***
## 4. Seq2seq recap: traininng our model

Now we are going to train our model is precisely same way as we trained several NMT models yesterday.

## *discuss* <-- How do we train our model? 

In [2]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('../utils')

In [3]:
from seq2seq import seq2seq
from chat_dataset import TextDataset, batchify
import torch
from tqdm import tqdm_notebook

In [4]:
# now we put everything on gpu

train_dataset = TextDataset('../data/convai2_simple_train_wpersona_with_starts.txt', device='cuda')
valid_dataset = TextDataset('../data/convai2_simple_valid_wpersona_with_starts.txt', device='cuda', dictionary=train_dataset.shared_dict)

In [5]:
from torch.utils.data import DataLoader
train_loader = DataLoader(train_dataset, batch_size=64, collate_fn=batchify, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=128, collate_fn=batchify, shuffle=False)

In [6]:
model = seq2seq(train_dataset.get_vocab_size(), train_dataset.get_vocab_size(), embedding_size=300, dropout=0.3, grad_clip=None, hidden_size=512, use_cuda=True, encoder_attention='dot', lr=0.01, encoder_shared_lt=False) 

In [7]:
model.load_model('./512_fasttext_dotatt_41.82977270882948.pt')

In [8]:
model

seq2seq(
  (decoder): DecoderRNN(
    (dropout): Dropout(p=0.3)
    (embedding): Embedding(18760, 300, padding_idx=0)
    (gru): GRU(300, 512, num_layers=2, batch_first=True, dropout=0.3)
    (attention): IBMAttentionLayer(
      (linear_out): Linear(in_features=1024, out_features=512, bias=False)
      (softmax): Softmax()
      (tanh): Tanh()
    )
    (out): Linear(in_features=512, out_features=18760, bias=True)
  )
  (encoder): EncoderRNN(
    (dropout): Dropout(p=0.3)
    (embedding): Embedding(18760, 300, padding_idx=0)
    (gru): GRU(300, 512, num_layers=2, batch_first=True, dropout=0.3)
  )
  (criterion): CrossEntropyLoss()
)

In [9]:
valid_loader = DataLoader(valid_dataset, batch_size=1, collate_fn=batchify, shuffle=False)

In [10]:
import ipdb
from tqdm import tqdm
import torch
for i, batch in enumerate(valid_loader):
    out_beam = model.eval_step(batch, decoding_strategy='beam:5')
    print('b {:.{prec}f} {}'.format(out_beam[0][1],train_dataset.pred2text(out_beam[0][0]),  prec=3))
    out_greedy = model.eval_step(batch, decoding_strategy='greedy')
    print('g {:.{prec}f}: {}'.format(out_greedy[0][1], train_dataset.pred2text(out_greedy[0][0]),  prec=3))
    print()
    
    #import ipdb; ipdb.set_trace()
    if i == 5:
        break

b -35.170 <sos> advisor one . alphabetize . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . to
g -34.217: <sos> debutante . alphabetize . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .

b -27.373 <sos> earrings . . to . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
g -37.857: <sos> dive . madonna . madonna . madonna . madonna . madonna . to . madonna . madonna . madonna . . . . . . . . . . . . . . . . . . . . .

b -29.090 <sos> earrings . . to . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
g -34.002: <sos> ax . to . to . madonna . madonna . madonna . . . . . . . . . . . . . . . . . . . . . . . . . . . . .

b -28.773 <sos> pet . to . with . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
g -38.777: <sos> dive . madonna . madonna . madonna . madonna . madonna . madonna . to . madonna . madonna . madonna . . . . . . . . . . . . . . . . . . .

b -27.246 <sos> pet . . . to . with . . . 

In [9]:
ipdb.pm()

> /home/kulikov/ammi-2019-nlp/utils/seq2seq.py(154)forward()
    153 
--> 154         attention_scores = torch.bmm(hid, encoder_output_t).squeeze(1)
    155 



ipdb>  u


> /opt/anaconda3/lib/python3.7/site-packages/torch/nn/modules/module.py(489)__call__()
    488         else:
--> 489             result = self.forward(*input, **kwargs)
    490         for hook in self._forward_hooks.values():



ipdb>  u


> /home/kulikov/ammi-2019-nlp/utils/seq2seq.py(210)forward()
    209             if self.attention is not None:
--> 210                 decoder_output_attended, attn_weights = self.attention(decoder_output, decoder_hidden, encoder_output, attention_mask)
    211                 output.append(decoder_output_attended)



...

ipdb>  u


> /home/kulikov/ammi-2019-nlp/utils/seq2seq.py(210)forward()
    209             if self.attention is not None:
--> 210                 decoder_output_attended, attn_weights = self.attention(decoder_output, decoder_hidden, encoder_output, attention_mask)
    211                 output.append(decoder_output_attended)



ipdb>  encoder_output.size()


torch.Size([1, 5, 512])


ipdb>  exit


***
## 5. Inference: how to extract a response from our model